In [1]:
import pandas as pd
import numpy as np
import math

from datetime import date
from datetime import datetime

import pprint as pp

import pyodbc

pd.options.display.max_columns = None

## Table of Contents

- [1 - Filter and Clean Data](#1)
- [2 - Rollup Class](#2)
  - [2.1 - Deploy Rollup](#2-1)
- [3 - Read in and Filter Supply Data](#3)
- [4 - NMCS Class](#4)
  - [4.1 - Run NMCS Calculator Class](#4-1)
- [5 - Final Join and Rollup for NMC and NMCS results](#5)
  - [5.1 - Deploy Join Class](#5-1)

<a name='1'></a>
## 1 - Filter and Clean Data

In [2]:
#read in IMDS data
# df = pd.read_csv('newCV22AFMaint3.csv')
df = pd.read_csv('lukeFeb.csv')

#filter to appropriate data
cols = ['EVT.EQUIP.ID','WCE.WUC.LCN.UNS','location','usmcsymbol','lewdId','Start_Date_0000','Stop_Date_Time','hours','NMCHours','PMCHours']
df = df[cols]

#Delete rows in which hours is negative
df = df.loc[df['NMCHours']>=0]

#rename cols for interpreatbility
df.rename({'Start_Date_0000':'Start_Date','EVT.EQUIP.ID':'Evt_ID','WCE.WUC.LCN.UNS':'Wuc'}, axis=1, inplace=True)

df['End_Date'] = pd.to_datetime(df['Stop_Date_Time']).dt.date

#remove all mafs where the start date is after end date
date_issues = df.loc[df['Start_Date']>df['Stop_Date_Time']]
df = df.loc[~(df['Start_Date']>df['Stop_Date_Time'])]

#drop all rows where event ID is NA
df.dropna(subset=['Evt_ID'], inplace=True)

#filter to appropriate dates
df = df.loc[df['Stop_Date_Time'] > '2021-12-01']

df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (34,35,54,58,59,60,62,67,68,69,70,71,72,73,76,77,81,82,83) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Evt_ID,Wuc,location,usmcsymbol,lewdId,Start_Date,Stop_Date_Time,hours,NMCHours,PMCHours,End_Date
0,A0025,540000,Cannon,NMC,5417-A220530101-1-1,2022-02-22,2022-02-23 09:00:00,38.0,38.0,0.0,2022-02-23
1,A0027,720000,Cannon,NMC,5417-A220320042-1-1,2022-02-01,2022-02-01 02:00:00,7.0,7.0,0.0,2022-02-01
2,A0027,270000,Cannon,NMC,5417-A220450238-1-1,2022-02-14,2022-02-23 06:00:00,227.0,227.0,0.0,2022-02-23
3,A0027,219000,Cannon,NMC,5417-A220460041-1-1,2022-02-15,2022-02-15 01:15:00,6.2,6.2,0.0,2022-02-15
4,A0027,219000,Cannon,NMC,5417-A220460041-2-1,2022-02-15,2022-02-15 01:20:00,6.3,6.3,0.0,2022-02-15


<a name='2'></a>
## 2 - Rollup Class

In [3]:
class NMC_Calculation():
    '''
    Rollup IMDS Data to calculate NMC Hours by Month and Lewd-ID.
    '''
    
    def __init__(self, df, status_detail, oor_overlay, monthly = True):
        
        self.df = df
        self.status_detail = status_detail
        self.oor_overlay = oor_overlay
        self.monthly = monthly
        
        
        
        self.NMCOverlay()
        self.ExpandDaily()
        self.OOROverlay()
        self.FinalRollup()
        
        
    
    def NMCOverlay (self):
        '''
        Overlay status detail to determine proper start date for each record.
        '''
        
        nmc = pd.read_csv(self.status_detail)
        nmc = nmc.iloc[:,1:]
    
        #convert our start and stop dates to datetime
        nmc['StartDateTime'] = pd.to_datetime(nmc['StartDateTime'])
        nmc['stop'] = pd.to_datetime(nmc['stop'])
        
        
        #match our status detail to IMDS by status detail and evt_id
        nmc['match'] = nmc['SerNum'].astype(str).str[-2:]
        self.df['match'] = self.df['Evt_ID'].astype(str).str[-2:]
        
        
        #Determine new start date base on NMC Flag
        
        #zip columns we need to compare to nmc columns
        self.df['zipped'] = [[i,j] for i,j in zip(self.df['match'], self.df['Stop_Date_Time'])]

        def adjust_nmc_start(x):
            match, End = x
            pdEnd = pd.to_datetime(End)


            match_idx = nmc.loc[(nmc['match']==match) & (nmc['StartDateTime']<pdEnd) & (nmc['stop']>pdEnd)].index
            if len(match_idx)>1:
                raise ValueError('Got more than one row.')
            elif len(match_idx)==0:
                return 'Okay'


            if nmc.iloc[match_idx]['NMCFlag'].values[0] == 0:
                return [(i,j,k) for i,j,k in zip(nmc.iloc[match_idx-2]['match'],nmc.iloc[match_idx-2]['NMCFlag'],nmc.iloc[match_idx-2]['stop'])]
            return [(i,j,k) for i,j,k in zip(nmc.iloc[match_idx-1]['match'],nmc.iloc[match_idx-1]['NMCFlag'],nmc.iloc[match_idx-1]['stop'])]

        
        self.df['nmc_zipped'] = self.df['zipped'].apply(lambda x: adjust_nmc_start(x))
        self.df['nmc_zipped'] = self.df['nmc_zipped'].apply(lambda x: x[0])
   
        # if a zipped match or nmc flag disagrees with our expectations, or is 'Okay', label them as 'Ok' to indicate using our original start date, and not zipped 'EndDate'
        self.df.loc[(self.df['nmc_zipped'].str[1]==1) | (self.df['nmc_zipped'].str[0]!=self.df['match'].astype(str)), 'nmc_zipped'] = 'Ok'
        self.df.loc[self.df['nmc_zipped']!='Ok','Start_Date'] = self.df['nmc_zipped'].apply(lambda x:x[-1])
        
        #drop our zipped columns
        self.df.drop([col for col in self.df.columns if 'zipped' in col],axis=1,inplace=True)
        
        
    
    def ExpandDaily(self):
        '''
        Expand each record to daily records between its start and end date, and calculate daily hours.
        '''
        #pp.pprint(self.df.head())
        # get Start date from new start date
        self.df.rename({'Start_Date':'Start_Date_Time'},axis=1, inplace=True)
        self.df = self.df.iloc[:,0:11]

        self.df['Start_Date'] = pd.to_datetime(self.df['Start_Date_Time']).dt.date
        
#         #avoid duplicate keys
#         if isinstance(self.df['Start_Date'], pd.DataFrame):
#             self.df['Start_Date'] = self.df['Start_Date'].astype(str).apply(
#                 pd.to_datetime, format='%Y-%m-%d', errors='coerce'
#             )
#         else:
#             self.df['Start_Date'] = pd.to_datetime(self.df['Start_Date_Time']).dt.date
        
        
        def drange(s, e):
            s = str(s)
            e = str(e)

            return pd.Series(pd.date_range(s,e))

        self.df['Date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(self.df['Start_Date']), pd.to_datetime(self.df['End_Date']))]
        self.df = self.df.explode('Date')
        
        #Calculate daily hours
        self.df['Datetime'] = self.df['Date'].astype(str)+' 00:00:00'
        self.df['Date'] = pd.to_datetime(self.df['Datetime']).dt.date
        self.df['end_bool'] = np.where(self.df['Date']==self.df['End_Date'], 'Yes','No')
        
        def daily_hour(row):
            if pd.to_datetime(row['Date'])==row['Start_Date']:
                return 24 - (int(str(row['Start_Date_Time'])[-8:-6]) + float(str(row['Start_Date_Time'])[-5:-3])/60)
            elif row['end_bool']=='Yes':
                return  int(str(row['Stop_Date_Time'])[-8:-6]) + (float(str(row['Stop_Date_Time'])[-5:-3])/60)
            else:
                return 24

        self.df['daily_hours'] = self.df.apply(lambda x: daily_hour(x), axis=1)
        self.df.drop('end_bool',axis=1,inplace=True)
        
        
    
    def OOROverlay(self):
        '''
        Determine whether aircraft went into OOR between start and end and substract that time from daily total. 
        '''
        
        oor = pd.read_csv(self.oor_overlay)
        oor = oor.iloc[:,1:]
        
        #create column based on last two numbers for 'SerNum' and 'EVT.EQUIP.ID' to overlay oor
        oor['match'] = oor['SerNum'].astype(str).str[-2:]
        self.df['match'] = self.df['Evt_ID'].astype(str).str[-2:]
        
        #filter out IR statuses in our oor df
        oor_df = oor.loc[oor['ReportingStatus']=='OOR']
        
        oor_df['StartDate'] = pd.to_datetime(oor_df['StartDateTime'],  errors='coerce')
        oor_df['EndDate'] = pd.to_datetime(oor_df['StopDateTime'],  errors='coerce')
        
        
        for i, row in oor_df.iterrows():
            self.df.loc[(self.df['match'] == row['match']) & (self.df['Date'].between(row['StartDate'], row['EndDate'])), 'oor_flag'] = 1

        self.df['oor_flag'].fillna(0, inplace=True)
        self.df['oor_flag'] = self.df['oor_flag'].astype(int)
        
        #remove days that are flagged as OOR
        self.df = self.df.loc[self.df['oor_flag']!=1]
        
    
    def FinalRollup(self):
        '''
        Depending on the keyword (monthly), we will roll up the data by month or lewd-id.
        '''
        
        #get first of every month to roll up to
        self.df['TransDate'] = self.df['Date'].to_numpy().astype('datetime64[M]')
        
        if self.monthly:
            self.df = self.df.groupby(['TransDate', 'Wuc','lewdId','usmcsymbol','Start_Date_Time','Stop_Date_Time'])['daily_hours'].sum().reset_index()
            self.df.dropna(inplace=True)
            
            self.df.rename({'daily_hours':'nmc_hours'},axis=1,inplace=True)
            self.df['nmc_hours'] = round(self.df['nmc_hours'],2)
            
        else:
            self.df = self.df.groupby(['lewdId','Evt_ID','Wuc','Start_Date_Time','Stop_Date_Time'])['daily_hours'].sum().reset_index()
            
            self.df.rename({'daily_hours':'nmc_hours'},axis=1,inplace=True)
            self.df['nmc_hours'] = round(self.df['nmc_hours'],2)
            

<a name='2-1'></a>
## 2.1 - Deploy Rollup

In [4]:
#monthly rollup
monthly = NMC_Calculation(df,'Lukestatusdetail2_Feb.csv','LukeOOR2_Feb.csv',monthly=True).df

#lewd rollup
# lewd = NMC_Calculation(df,'Lukestatusdetail.csv','LukeOOR.csv',monthly=False).df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
monthly.head()

,TransDate,Wuc,lewdId,usmcsymbol,Start_Date_Time,Stop_Date_Time,nmc_hours
0,2017-06-01,56100111,5932-A171570030-1-1,NMC,2017-06-06,2021-10-01 09:00:00,582.9
1,2017-06-01,56100226,5932-A171570031-1-1,NMC,2017-06-06,2021-10-01 08:00:00,582.9
2,2017-07-01,56100111,5932-A171570030-1-1,NMC,2017-06-06,2021-10-01 09:00:00,744.0
3,2017-07-01,56100226,5932-A171570031-1-1,NMC,2017-06-06,2021-10-01 08:00:00,744.0
4,2017-08-01,56100111,5932-A171570030-1-1,NMC,2017-06-06,2021-10-01 09:00:00,744.0


<a name='3'></a>
## 3 - Read in and Filter Supply Data

In [5]:
cnxn_str = ("Driver={SQL Server};"
            "Server=NRS0752PXSQ07V\DW02;"
            "Database=WAREHOUSEDB2;"
           "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)

# cursor = cnxn.cursor()

# cursor.execute("declare @startdate as datetime;")
# cursor.execute("set @startdate = DATEADD(day, -5, Convert(date, getdate()))")

try:
    df = pd.read_sql("SELECT * FROM AIRFORCE.PartsOrdered;", cnxn)
#     df = pd.read_sql("SELECT * FROM AIRFORCE.AFPartsOrdered75months WHERE [TR DATE] BETWEEN '2016-01-01' AND '2017-01-01';", cnxn)
    del cnxn
except:
    del cnxn
df.shape

(50235, 24)

In [5]:
# df = pd.read_csv('Parts_Ordered_Oct_21.csv')

In [6]:
cols = {col:col.lower() for col in df.columns}
df.rename(cols, axis=1, inplace=True)

print(df.shape)
pri_remove =['BQ','CZ','','AG','BG','BZ','CQ','BT','Y','JC','I','1C','1G','BC','N']
df = df.loc[~(df['pri'].isin(pri_remove))]
print(df.shape)

(50235, 24)
(25072, 24)


<a name='4'></a>
## 4 - NMCS Class

In [7]:
class NMCSCalculator():
    '''
    Determine NMCS hours using AF Supply Data.
    '''
    
    def __init__(self, df, monthly = False):
        self.df = df
        self.jcn_sums = {}
        
        if monthly == True:
            self.docDateExtract()
            self.SiAssigns()
            self.filterData()
            self.retrieveRangeSums(keep_dates=True)
            self.expandDaily()
        else:
            self.docDateExtract()
            self.SiAssigns()
            self.filterData()
            self.retrieveRangeSums()
        
        
        
    def docDateExtract(self):
        
        self.df['doc_date'] = self.df['document'].str[6:10].apply(lambda x: '202'+x if eval(x[0])<3 else '201'+x)

        julian = self.df.doc_date.str[4:].str.extract("([1-9][0-9]?[0-9]?)")    
        self.df["doc_date"] = self.df.doc_date.str[:4] + "-" + julian.iloc[:,0].astype(str)
        self.df['doc_date'] = pd.to_datetime(self.df['doc_date'], errors='coerce', format='%Y-%j')
        
        null_docs = self.df.loc[self.df['doc_date'].isna()]['document'].unique()
        
        for doc in null_docs:
            self.df.loc[(self.df['document']==doc) & (self.df['doc_date'].isnull()), 'doc_date'] = self.df.loc[self.df['document']==doc]['tr date'].min()
         
        self.df.loc[pd.to_datetime(self.df['doc_date']).dt.is_leap_year, 'doc_date'] = pd.to_datetime(self.df['doc_date']) - pd.DateOffset(days=1)
        
        odd_docs = self.df.loc[pd.to_datetime(self.df['doc_date']).dt.date > pd.to_datetime(self.df['tr date']).dt.date]['document'].unique()
        
        for doc in odd_docs:
            self.df.loc[self.df['document']==doc, 'doc_date'] = self.df.loc[self.df['document']==doc]['tr date'].min()
            
            
            
    def SiAssigns(self):
        #Assignning JCNs
        
        #turn empty jcn values to Nan values
        self.df.loc[self.df['jcn'].str.strip()=='', 'jcn'] = np.nan
        #fill our Nan values with the appropriate JCN
        self.df.update(self.df.groupby('document')['jcn'].fillna(lambda x: x.fill().bfill()))
        
        #Assigning Mark-For
        
        #grab the first occurence of an 'A' in mark for - for each unique document
        first = self.df.loc[self.df['mark for'].astype(str).str[2]=='A'].sort_values(self.df.columns.tolist())\
                  .drop_duplicates(subset=['document'], keep='first')
        
        #populate 1SI entries with 'A' mark for values
        for i, row in first.iterrows():
            self.df.loc[self.df['document']==row['document'], 'mark for'] = row['mark for']
        
       
            
            
    
    def filterData(self):
        #remove all that are not 1SI or DOC, and remove all non-aircrafts if not AOO
        self.df = self.df.loc[(self.df['tric'].isin(['1SI','DOC']))] 
        self.df = self.df.loc[(self.df['mark for'].str[2]=='A')]

        #strip away white spaces from jcn
        self.df['jcn'] = self.df['jcn'].astype(str).str.strip()
        #remove erroneous jcn values
        self.df = self.df.loc[self.df['jcn'].str[-1].isin(['1','2','3','4','5','6','7','8','9'])]
        
        #Drop Duplicates
        self.df.drop_duplicates(subset=['jcn','doc_date','tr date'], inplace=True)
        
        
        self.df.reset_index(inplace=True)
        self.df.drop('index', axis=1,inplace=True)
    
        
        
    def createDocLog(self):
        
        
        def hourConvert(row):
            
            minute_delta = pd.Timedelta(pd.to_datetime(row['tr date'])-pd.to_datetime(row['doc_date'])).total_seconds()
            return minute_delta / 3600.0
            
        self.df['hours'] = np.where(pd.to_datetime(self.df['doc_date'])==pd.to_datetime(self.df['tr date']), 3, self.df.apply(hourConvert, axis=1))
#                                     self.df.apply(lambda x:  pd.Timedelta(pd.datetime(x['tr date'])-pd.datetime(x['doc_date'])).total_seconds() / 3600.0))
#                                     pd.Timedelta(pd.datetime(self.df['tr date'])-pd.datetime(self.df['doc_date'])).total_seconds() / 3600.0)
        self.df['doc_date'] = pd.to_datetime(self.df['doc_date']).dt.date
    
    
    def retrieveRangeSums(self, keep_dates=False):
        
        #filter to appropriate columns
        self.df = self.df[['sran','jcn','doc_date','tr date']].sort_values(by=['jcn','doc_date'])
        
        #covert to datetime objects
        self.df['doc_date'] = pd.to_datetime(self.df['doc_date']).dt.date
        self.df['tr date'] = pd.to_datetime(self.df['tr date']).dt.date
        
        #create dict of each jcn possible date ranges
        jcn_dict = {}

        for i, row in self.df.iterrows():
            if (row['jcn'], row['sran']) in jcn_dict:
                jcn_dict[(row['jcn'], row['sran'])].append([row['doc_date'], row['tr date']])
            else:
                jcn_dict[(row['jcn'], row['sran'])] = [[row['doc_date'], row['tr date']]]
                
                
                
        #filter date ranges to the longest span
        from datetime import datetime
        from collections import namedtuple
        
        Range = namedtuple('Range', ['start', 'end'])
        jcn_ranges = {}

        for key, value in jcn_dict.items():
            earliest_start, latest_end = value[0][0], value[0][1]
            extra = []
            for start, end in value[1:]:
                if ((start <= latest_end) and (start >= earliest_start)) or ((end >= latest_end) and (end <= earliest_start)):
                    r1 = Range(start=start, end=end)
                    r2 = Range(start=earliest_start, end=latest_end)
                    earliest_start = min(r1.start, r2.start)
                    latest_end = max(r1.end, r2.end)
                else:
                    extra.append((start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')))
                    earliest_start = start
                    latest_end = end
            jcn_ranges[key] = [(earliest_start.strftime('%Y-%m-%d'), latest_end.strftime('%Y-%m-%d'))]
            if extra:
                jcn_ranges[key].extend(extra)
        jcn_ranges = {k:set(v) for k,v in jcn_ranges.items()}

        
        if keep_dates:            
            self.df =pd.DataFrame.from_dict(jcn_ranges, orient='index').reset_index()
            
            # melt all date columns
            max_dates = max([col for col in self.df.columns if str(col).isdigit()])
            self.df = pd.melt(self.df, id_vars='index', value_vars=list(range(max_dates+1)), value_name='Dates')

            # drop Null values
            self.df.dropna(inplace=True)

            #unzip jcn/sran and StartDate/EndDate
            self.df['jcn'], self.df['sran'] = zip(*self.df['index'])
            self.df['StartDate'], self.df['EndDate'] = zip(*self.df['Dates'])

            #drop 'index', 'Dates' and 'variable' columns
            self.df.drop(['index','variable','Dates'], axis=1, inplace=True)   
            
        else:
            #get date deltas in hours based on each start date and end date pair
            for key, val in jcn_ranges.items():
                total_hours = 0
                for start, end in val:
                    if start==end:
                        total_hours+=3
                    else:
                        hour_delta = pd.Timedelta(pd.to_datetime(end)-pd.to_datetime(start)).total_seconds() / 3600.0
                        total_hours+=hour_delta
                self.jcn_sums[key] = total_hours

            self.df = pd.DataFrame(self.jcn_sums.items(), columns = ['jcn','nmcs_hours'])
            self.df['jcn'], self.df['sran'] = zip(*self.df['jcn'])
          

            
            
    def expandDaily(self):
        def drange(s, e):
            s = str(s)
            e = str(e)

            return pd.Series(pd.date_range(s,e))

        self.df['Date'] = [pd.date_range(s, e, freq='d') for s, e in
              zip(pd.to_datetime(self.df['StartDate']), pd.to_datetime(self.df['EndDate']))]
        
        #expand rows for each day
        self.df = self.df.explode('Date')
        
        def daily_hour(row):
            if row['StartDate']==row['EndDate']:
                return 3
            elif row['Date']==row['EndDate']:
                return 0
            else:
                return 24

        self.df['daily_hours'] = self.df[['StartDate','EndDate','Date']].astype(str).apply(lambda x: daily_hour(x),axis=1)
        
        #get first of every month to roll up to when needed
        self.df['TransDate'] = self.df['Date'].to_numpy().astype('datetime64[M]')
        
        #create final groupby object
        self.df = self.df.groupby(['TransDate','jcn','sran'])['daily_hours'].sum().reset_index()
        self.df.rename({'daily_hours':'nmcs_hours'},axis=1,inplace=True)
        self.df['nmcs_hours'] = self.df['nmcs_hours'].astype(float)
    

<a name='4-1'></a>
## 4.1 - Run NMCS Calculator Class

In [8]:
df_monthly = NMCSCalculator(df, monthly=True).df
# df_lewd = NMCSCalculator(df).df

###### <a name='5'></a>
## 5 - Final Join and Rollup for NMC and NMCS results

In [9]:
class NMCJoin:
    '''
    Join our NMC results with our NMCS results by lewd-Id and monthly.
    '''
    def __init__(self, nmc_df, nmcs_df, monthly = True):
        self.nmc_df = nmc_df
        self.nmcs_df = nmcs_df
        self.monthly = monthly
        
        self.final_df = pd.DataFrame()
    
    
    
        self.NMCPrep()
        self.NMCSPrep()
        self.FinalRollup()
    
    
    def NMCPrep(self):
        '''
        Prepare our NMC data for merger.
        '''
        
        sran_map = {
            '5417':'1801',
            '5932':'1820',
            '7070':'2316',
            '5932':'4417',
            '5955':'4469',
            '5417':'4855',
            '7070':'5209',
            '5518':'5270',
            '7476':'5518',
            '5417':'multi-sran', #('7599','5807','5819','5897','5908','4804','5820','5806')
            '5932':'multi-sran', #('7599','5807','5819','5897','5908','4804','5820','5806')
            '7476':'multi-sran', #('7599','5807','5819','5897','5908','4804','5820','5806')
            '5955':'6311'
        }
        
        #sum all hours to ddr==1
        self.nmc_df['non_ddr'] = self.nmc_df['lewdId'].str.rsplit('-', 1).str.get(0)
        
       
    # if self.monthly:
        #group our lewdID by ddr
        non_ddr = self.nmc_df.groupby(['TransDate', 'Wuc','non_ddr','Start_Date_Time','Stop_Date_Time'])['nmc_hours'].sum().reset_index()

         #filter to where that last of our lewdID (ddr) is 1
        self.nmc_df = self.nmc_df.loc[self.nmc_df['lewdId'].str.endswith('1')]
        self.nmc_df.drop('nmc_hours',axis=1,inplace=True)
        self.nmc_df = pd.merge(self.nmc_df, non_ddr, how='left',on=['TransDate', 'Wuc','non_ddr','Start_Date_Time','Stop_Date_Time'])
        self.nmc_df.drop('non_ddr',axis=1,inplace=True)
    
            
# if-else this statement if needed
            
#             #group our lewdID by ddr
#             non_ddr = self.nmc_df.groupby(['non_ddr'])['nmc_hours'].sum().reset_index()
            
#             #filter to where that last of our lewdID (ddr) is 1
#             self.nmc_df = self.nmc_df.loc[self.nmc_df['lewdId'].str.endswith('1')]
            
            
            
        # get location of lewd to match to sran
        self.nmc_df['location'] = self.nmc_df['lewdId'].str.split('-').str.get(0)

        #get the sran from the location, to map to the nmcs
        self.nmc_df['sran_map'] = self.nmc_df['location'].astype(str).map(sran_map) 

        #create jcn from lewd_id column
        self.nmc_df['id'] = self.nmc_df['lewdId'].str[5:-2]
        
        #create evt_id column
        self.nmc_df['evt_id'] = self.nmc_df['id'].str.split('-').str.get(0)
        
        
        
    def NMCSPrep(self):
        '''
        Prepare our NMCS data for merger
        '''
        import string
        import re

        #since locations can map to multiple srans, let's change our srans to their appropriate list group
        sran_group = ['7599','5807','5819','5897','5908','4804','5820','5806']
        
        #rename 'sran' to 'sran_map' for merging purposes
        self.nmcs_df.rename({'sran':'sran_map'},axis=1,inplace=True)
        
        #translate our srans
        self.nmcs_df.loc[self.nmcs_df['sran_map'].astype(str).isin(sran_group), 'sran_map'] = 'multi-sran'
        
        #getting rid of weird JCN entries
        self.nmcs_df = self.nmcs_df.loc[~(self.nmcs_df['jcn'].str[-3:].str.contains('|'.join(string.ascii_lowercase), flags=re.IGNORECASE))]
        
        #create id from nmv column
        self.nmcs_df['id'] = self.nmcs_df['jcn'].astype(str).apply(lambda x: x[:-3]+'-'+str(int(x[-3:])))

        #turn sran col into str before merger
        self.nmcs_df['sran_map'] = self.nmcs_df['sran_map'].astype(str)
        
        #create event id for merger
        self.nmcs_df['evt_id'] = self.nmcs_df['id'].str.split('-').str.get(0)
        
        
        
    
    def FinalRollup(self):
        '''
        Now that our dataframes are properly formatted, we can now join them and group by the appropriate field (month/lewd).
        '''
        #ensure transdates are the same datas type for merger
        self.nmcs_df['TransDate'] = self.nmcs_df['TransDate'].astype(str)
        self.nmc_df['TransDate'] = self.nmc_df['TransDate'].astype(str)
        
        
        self.final_df = pd.merge(self.nmc_df, self.nmcs_df, on=['TransDate','evt_id','sran_map'], how='left')   

         #if nmcs hours is greater than nmc hours, set nmcs hours to nmc hours
        self.final_df['nmcs_hours'].fillna(0, inplace=True)
        self.final_df['nmcs_hours'] = np.where(self.final_df['nmcs_hours']>self.final_df['nmc_hours'],self.final_df['nmc_hours'],self.final_df['nmcs_hours'])
    
        
        if self.monthly:    

            #final groupby statement (add back sran map)
            self.final_df = (self.final_df.groupby(['TransDate','Wuc','lewdId','usmcsymbol','Start_Date_Time','Stop_Date_Time','evt_id','sran_map'])
                           [['nmc_hours','nmcs_hours']]
                           .sum()
                           .reset_index())
            
    
            # Rollup to month and Wuc
            self.final_df = (self.final_df.groupby(['TransDate','Wuc','usmcsymbol'])[['nmc_hours','nmcs_hours']]
                           .sum()
                           .reset_index())
            
            
        else:
            

            #reorganize columns
            self.final_df = self.final_df[['lewdId','evt_id','Wuc','usmcsymbol','Start_Date_Time','Stop_Date_Time', 
                                           'jcn','sran_map','location', 'nmc_hours','nmcs_hours']]

            #group by lewdId
            self.final_df = (self.final_df.groupby(['lewdId','evt_id','Wuc','usmcsymbol','Start_Date_Time','Stop_Date_Time',
                                                     'sran_map','location'])[['nmc_hours','nmcs_hours']]
                           .sum()
                           .reset_index())

            
        #####FULL VERIONS#########
#         #if nmcs hours is greater than nmc hours, set nmcs hours to nmc hours
#         self.final_df['nmcs_hours'].fillna(0, inplace=True)
#         self.final_df['nmcs_hours'] = np.where(self.final_df['nmcs_hours']>self.final_df['nmc_hours'],self.final_df['nmc_hours'],self.final_df['nmcs_hours'])
#         print(self.final_df.nmc_hours.sum())
        self.df = self.final_df

<a name='5-1'></a>
## 5.1 - Deploy Join Class

In [10]:
monthly_rollup = NMCJoin(monthly, df_monthly, monthly=True).df
lewd_rollup = NMCJoin(monthly, df_monthly, monthly=False).df

C:\Users\Luke.Gray\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Luke.Gray\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
print('finito')

finito


In [15]:
# monthly_rollup.to_csv('2016_monthly_rollup.csv', index=False)
# lewd_rollup.to_csv('2016_lewd_rollup.csv', index=False)

## Concat All Years

In [16]:
import glob

monthly = glob.glob('*monthly_rollup.csv')
lewd = glob.glob('*lewd_rollup.csv')

monthly_li, lewd_li = [],[]

for file in monthly:
    df = pd.read_csv(file)
    monthly_li.append(df)
    
for file in lewd:
    df = pd.read_csv(file)
    lewd_li.append(df)
    
month_fin = pd.concat(monthly_li)
lewd_fin = pd.concat(lewd_li)

## Pivot Monthly

In [12]:
#pivot our nmc/pmc rows
months = pd.pivot_table(monthly_rollup, values=['nmc_hours','nmcs_hours'], 
                        index=['TransDate','Wuc'], columns='usmcsymbol',
                        aggfunc=np.sum)

In [13]:
#flatten multi-index
months.columns = months.columns.get_level_values(0)

#rename columns
months.columns.values[0] = "nmc_hours"
months.columns.values[1] = "pmc_hours"
months.columns.values[2] = "nmcs_hours"
months.columns.values[3] = "pmcs_hours"

months.reset_index(inplace=True)

In [14]:
#fill Nan with 0
months.fillna(0, inplace=True)

#final_groupby
months = (months.groupby(['TransDate','Wuc'])[['nmc_hours','pmc_hours','nmcs_hours','pmcs_hours']]
          .sum()
          .reset_index())

In [15]:
#Sanity Check
months.loc[(months['nmc_hours']>0) & (months['pmc_hours']>0)].head()

,TransDate,Wuc,nmc_hours,pmc_hours,nmcs_hours,pmcs_hours
59,2021-09-01,281100,789.36,65.78,0.0,0.0
76,2021-09-01,281401,460.46,65.78,0.0,0.0
88,2021-09-01,2823AK,197.34,65.78,0.0,0.0
164,2021-10-01,281100,864.00,72.00,0.0,0.0
181,2021-10-01,281401,504.00,72.00,0.0,0.0


## Pivot Lewd

In [16]:
#pivot our nmc/pmc rows
lewds = pd.pivot_table(lewd_rollup, values=['nmc_hours','nmcs_hours'], 
                        index=['lewdId','evt_id','Wuc','Start_Date_Time',
                               'Stop_Date_Time','sran_map','location'], columns='usmcsymbol',
                        aggfunc=np.sum)

In [17]:
#flatten multi-index
lewds.columns = lewds.columns.get_level_values(0)

#rename columns
lewds.columns.values[0] = "nmc_hours"
lewds.columns.values[1] = "pmc_hours"
lewds.columns.values[2] = "nmcs_hours"
lewds.columns.values[3] = "pmcs_hours"

lewds.reset_index(inplace=True)

In [18]:
#fill Nan with 0
lewds.fillna(0, inplace=True)

#final_groupby
lewds = (lewds.groupby(['lewdId','evt_id','Wuc','Start_Date_Time',
                               'Stop_Date_Time','sran_map','location'])
         [['nmc_hours','pmc_hours','nmcs_hours','pmcs_hours']]
          .sum()
          .reset_index())

In [19]:
lewds.sample(5)

,lewdId,evt_id,Wuc,Start_Date_Time,Stop_Date_Time,sran_map,location,nmc_hours,pmc_hours,nmcs_hours,pmcs_hours
1206,5417-A213080058-2-1,A213080058,052060000030A,2021-11-30 04:31:00,2021-12-07 11:27:48,multi-sran,5417,126.93,0.0,0.0,0.0
11009,7070-C213360161-289-1,C213360161,2911SI,2021-11-21 21:15:00,2021-12-23 09:00:00,5209,7070,755.75,0.0,0.0,0.0
5438,5932-A213220071-14-1,A213220071,6221130531,2021-11-24 21:10:00,2021-12-07 11:27:48,multi-sran,5932,302.28,0.0,0.0,0.0
9714,5955-A220240103-3-1,A220240103,245001A7,2022-01-24 00:00:00,2022-01-24 18:00:00,6311,5955,1.98,0.0,0.0,0.0
10199,7070-C213070125-1-1,C213070125,300000,2021-12-08 13:20:00,2021-12-11 12:20:00,5209,7070,0.00,71.0,0.0,0.0


## Export

In [19]:
months.to_csv('AFMonthlyRollup_Feb.csv',index=False)
lewds.to_csv('AFLewdRollup_Feb.csv',index=False)

In [ ]:
    #put before roll up if needed       
    ###fixing hours
            #for those id's that apply to multiple lewd ids, use the sran to divvy up nmcs hours
#             self.final_df['dup_zipped'] = [[i,j] for i,j in zip(self.final_df['evt_id'],self.final_df['sran_map'])]

#             id_dups = self.final_df.loc[(self.final_df.groupby('evt_id')['lewdId'].transform('nunique').gt(1)) &
#                                  (self.final_df['nmc_hours'].notnull())]['dup_zipped']
            
            
#             #create groupby so we can reassign according to sran
#             df_for_sran = self.nmcs_df.groupby(['evt_id','sran_map'])['nmcs_hours'].sum().reset_index()
            
#             for idx, sran in id_dups:
#                 self.final_df.loc[(self.final_df['evt_id']==idx) & (self.final_df['sran_map']==sran), 
#                            'nmcs_hours'] = df_for_sran.loc[(df_for_sran['evt_id']==idx) & (df_for_sran['sran_map']==sran)]['nmcs_hours']
                
#             #drop unnecessary columns
#             self.final_df.drop('dup_zipped', axis=1,inplace=True)
            #####fin fixing hours